# IoT and sensor data
## From "pdm_task.ipynb" February 19, 2025
(use venv_requirements.txt)

i. The first one is explained in bullet 2 of slide 16 of the slide deck “IoT & Sensor
Data.pptx”. You need to do the entire task described in bullet 2 of slide 16.

# Monitoring IoT Sensor Data
## Data Subscriber Simulation

This part of code simulates an IoT data subscriber that receives data sent out by a publisher via a MQTT broker. Note a subscriber doesn't work without a publisher or a broker.

See IoT_Publisher.ipynb for more details about the publisher and broker.

## 4. Modification of the MQTT example

In [ ]:
import paho.mqtt.client as mqtt
import json
from sklearn.neighbors import KNeighborsClassifier

# initialization of variables to be used
last_5_records = []
correct_classifications = 0
total_classifications = 0

def on_connect(client, userdata, flags, rc):
    client.subscribe('diabetes')

# This function defines what to do when we receives a message from the brokder
def on_message(client, userdata, msg):
    global last_5_records, correct_classifications, total_classifications

    #Defining 3KNN Classifier 
    knn = KNeighborsClassifier(n_neighbors=3)

    #decoding payload
    payload = msg.payload.decode()

    #deserialize received string into dictionary
    record = json.loads(payload)    
    
    #extract features and labels
    features = [record['Glucose'], record['BloodPressure'], record['Insulin']]
    label = record['Outcome']
    
    
    last_5_records.append((features, label))

    # first in, first out
    if len(last_5_records) > 5:
        last_5_records.pop(0)

    # If we have at least 5 records, we perform 3NN classification
    if len(last_5_records) == 5:
        #features
        X_train = [rec[0] for rec in last_5_records[:-1]]
        #labels
        y_train = [rec[1] for rec in last_5_records[:-1]]

        knn.fit(X_train, y_train)

        #[0] = the first element returned by predict method, as we are only running predict on one sample at a time. 
        predicted_label = knn.predict([features])[0]

        #counting our correct_classifications        
        if predicted_label == label:
            correct_classifications += 1
        #and adding to how many classifications we have done so far 
        total_classifications +=1
        print(f"Total classifications: {total_classifications}, Correct classifications: {correct_classifications}")

        # the backstop so we dont exceed 1000 -
        if total_classifications >= 999:
            print(f"Final accuracy: {correct_classifications / total_classifications}")
            total_classifications =0
            correct_classifications = 0

            
            #if we want to dc just uncomment line below, but the task asked us to do this contninously so we loop forever.
            #client.disconnect()

#below is code from taken from IoT_subscriber.ipynb
mqttc = mqtt.Client()
mqttc.on_connect = on_connect
mqttc.on_message = on_message
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)
mqttc.loop_forever()